# Analysing JSON Data

This example demonstrates how to download and process [JSON](https://en.wikipedia.org/wiki/JSON) datasets using the open source data transformation tool: [Arc](https://arc.tripl.ai/). In this example the data is downloaded directly from the source using an Arc [HTTPExtract](https://arc.tripl.ai/extract/#httpextract) stage but the data could equally have been downloaded prior to the job.

This example aims to show how to:

1. source data using the [HTTPExtract](https://arc.tripl.ai/extract/#httpextract) stage.
1. parse a [JSON](https://en.wikipedia.org/wiki/JSON) response using the [JSONExtract](https://arc.tripl.ai/extract/#jsonextract) stage.
1. process the nested result into a standard tabular representation using the [SQLTransform](https://arc.tripl.ai/transform/#sqltransform) stage with the inline `%sql` 'magic' functionality
1. ensure data quality and load assurance needs are met using the [SQLValidate](https://arc.tripl.ai/validate/#sqlvalidate) stage.
1. write out the data so it can be consumed by other people or jobs.

This example is possible due to data provided by the excellent [Australian Bureau of Meteorology](http://www.bom.gov.au).

### Define variables

To make this process reusable we first define a variable which we can use to dynamically replace the weather station identifier at execution time with a different station. This makes this job reusable for all [Australian Bureau of Meteorology](http://www.bom.gov.au) weather stations.

To set a variable to use when developing logic it can be set with the `%env` magic:

In [ ]:
%env
WMO_STATION_ID=94768

WMO_STATION_ID: 94768

### Download the weather data

This step uses the [HTTPExtract](https://arc.tripl.ai/extract/#httpextract) stage to directly download the data at `http://www.bom.gov.au/fwo/IDN60901/IDN60901.94768.json` and makes that dataset available with the alias `weather_raw` (defined by `outputView`). The reponse is the raw response data to allow you to choose how to process the data.

Here we are using the `WMO_STATION_ID` variable and [string interpolation](https://en.wikipedia.org/wiki/String_interpolation) to call the endpoint based on the provided station identifier.

In [ ]:
%arc truncate=100
{
  "type": "HTTPExtract",
  "name": "download weather data",
  "environments": [
    "production",
    "test"
  ],
  "inputURI": "http://www.bom.gov.au/fwo/IDN60901/IDN60901."${WMO_STATION_ID}".json",
  "outputView": "weather_raw",
  "persist": true
}

5/5

uri,statusCode,reasonPhrase,contentType,contentLength,body
http://www.bom.gov.au/fwo/IDN60901/IDN60901.94768.json,200,OK,Content-Type: application/json,127333,"{ ""observations"": { ""notice"": [ { ""copyright"": ""Copyright Commonwealth of Australia 202..."


### Parse the response

This step uses the [JSONExtract](https://arc.tripl.ai/extract/#jsonextract) stage to parse data in the `body` field (defined by `inputField`) of the incoming `weather_raw` dataset (defined by `inputView`) and makes that dataset available with the alias `weather_nested` (defined by `outputView`).

If the `IDN60901.94768.json` file had been downloaded prior to this job (instead of being directly downloaded using the [HTTPExtract](https://arc.tripl.ai/extract/#httpextract) stage) then `inputView` and `inputField` would be replaced with an `inputURI` pointing to that file or multiple files.

In [ ]:
{
  "type": "JSONExtract",
  "name": "parse weather data http response",
  "environments": [
    "production",
    "test"
  ],
  "inputView": "weather_raw",
  "inputField": "body",
  "outputView": "weather_nested"
}

205/205

observations,_filename,_index
"[[[20210325010000, 26.4, 27.1, -,,, -,, 6.4, 16...",,1


### View the schema

Arc runs on top of [Apache Spark](https://spark.apache.org/) which supports advanced data types such as nested objects inside arrays like returned in the `IDN60901.94768.json` [JSON](https://en.wikipedia.org/wiki/JSON) dataset in addition to the standard `string`, `float` and `date` data types. 

To see the schema and help write queries to extract the data the `%printschema` magic can be used to show the layout of the data within the parsed [JSON](https://en.wikipedia.org/wiki/JSON) response.

In [ ]:
%printschema
weather_nested

root
 |-- observations: struct (nullable = true)
 |    |-- data: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- aifstime_utc: string (nullable = true)
 |    |    |    |-- air_temp: double (nullable = true)
 |    |    |    |-- apparent_t: double (nullable = true)
 |    |    |    |-- cloud: string (nullable = true)
 |    |    |    |-- cloud_base_m: string (nullable = true)
 |    |    |    |-- cloud_oktas: string (nullable = true)
 |    |    |    |-- cloud_type: string (nullable = true)
 |    |    |    |-- cloud_type_id: string (nullable = true)
 |    |    |    |-- delta_t: double (nullable = true)
 |    |    |    |-- dewpt: double (nullable = true)
 |    |    |    |-- gust_kmh: string (nullable = true)
 |    |    |    |-- gust_kt: string (nullable = true)
 |    |    |    |-- history_product: string (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- local_date_time: string (nullable = true)
 |    |    |

### Explode the data

[Arc](https://arc.tripl.ai/) aims to help business users safely and independently build their own data processing jobs by removing the multiple translations traditionally required when having to hand off the work to developers or other 'go-between' roles.

To do this we need a way of those users expressing business intent for which we employ [SQL](https://en.wikipedia.org/wiki/SQL) as a dialect (which is why we say that [Arc](https://arc.tripl.ai/) is 'SQL First'). We have found that SQL supports most standard data transformation requirements, is relatively easy to learn and easy to hire for.

The statement below demonstrates three key operations for processing the `IDN60901.94768.json` nested dataset:

1. use of the [POSEXPLODE](https://spark.apache.org/docs/latest/api/sql/index.html#posexplode) SQL function to separates the elements of the `observations.data` into multiple rows. [POSEXPLODE](https://spark.apache.org/docs/latest/api/sql/index.html#posexplode) also returns the `position` of the data (i.e. the index) in the array which can be useful if the position in the array is important. There is also a most simplistic [EXPLODE](https://spark.apache.org/docs/latest/api/sql/index.html#explode) function which does not return the position.
1. use of a subquery to turn the data returned by [POSEXPLODE](https://spark.apache.org/docs/latest/api/sql/index.html#posexplode) into a normal tabular representation by selecting the required fields. If desired the use of `observation.*` would also work instead of selecting individual fields.
1. parsing the `aifstime_utc` field into a [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time) timestamp which can then be safely used to order the data.

In [ ]:
%sql name="calculate weather" outputView=weather environments=production,test persist=true
SELECT
  -- convert aifstime_utc to a timestamp object
  MAKE_TIMESTAMP(
    SUBSTR(observation.aifstime_utc, 0, 4),
    SUBSTR(observation.aifstime_utc, 5, 2),
    SUBSTR(observation.aifstime_utc, 7, 2),
    SUBSTR(observation.aifstime_utc, 9, 2),
    SUBSTR(observation.aifstime_utc, 11, 2),
    SUBSTR(observation.aifstime_utc, 13, 2),
    'UTC'
  ) AS timestamp
  ,observation.air_temp
  ,observation.apparent_t
  ,observation.delta_t
  ,observation.dewpt
  ,observation.press
  ,observation.wind_spd_kmh
  ,observation.history_product
  ,observation.wmo
  ,header.refresh_message
  ,_index
FROM (
  SELECT
    POSEXPLODE(observations.data) AS (_index, observation)
  FROM weather_nested
) observations

CROSS JOIN

(
  SELECT
    EXPLODE(observations.header) AS header
  FROM weather_nested
) header  

6/6

timestamp,air_temp,apparent_t,delta_t,dewpt,press,wind_spd_kmh,history_product,wmo,refresh_message,_index
2021-03-25 01:00:00Z,26.40,27.10,6.40,16.10,1003.90,7,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,0
2021-03-25 00:30:00Z,26.20,26.90,6.20,16.20,1003.90,7,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,1
2021-03-25 00:00:00Z,25.10,25.40,5.60,16.00,1003.80,9,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,2
2021-03-24 23:30:00Z,23.90,23.80,4.90,15.90,1003.80,11,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,3
2021-03-24 23:00:00Z,21.60,22.40,3.40,15.90,1003.80,6,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,4
2021-03-24 22:30:00Z,21.30,22.00,3.20,16.00,1003.70,7,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,5
2021-03-24 22:00:00Z,18.90,20.10,2.30,15.00,1003.50,2,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,6
2021-03-24 21:30:00Z,18.10,19.50,2.10,14.40,1003.00,0,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,7
2021-03-24 21:00:00Z,17.60,17.70,2.10,13.90,1002.40,6,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,8
2021-03-24 20:30:00Z,17.30,17.40,2.00,13.80,1002.20,6,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,9


### Validate the data / load assurance

Before using the data it is a good idea to ensure that certain data quality guarantees can be provided to downstream consumers of the data. To do this we once again use an [Arc](https://arc.tripl.ai/) 'SQL First' approach to define data quality rules using a [SQLValidate](https://arc.tripl.ai/validate/#sqlvalidate) stage. When building [Arc](https://arc.tripl.ai/) we could have tried to impose a set of standardised business rules but inevitably someone would have a case which was not covered but can be met using a SQL statement.

The statement below demonstrates three key operations for our processed the `IDN60901.94768.json` flattened dataset:

1. to apply individual rules a good method is to use case statements. For example if we want to ensure that all the timestamp values are populated we can write a statement to find any missing values like: `CASE WHEN timestamp IS NULL THEN 1 ELSE 0 END AS timestamp_null`. `CASE` statements allow very detailed business rules to be defined. Many of these rules can be quickly added for different conditions you care about.
1. once the individual rules have been applied we need to define what conditions need to be met for the data to be considered to have met data quality guarantees (and return a `TRUE`/`FALSE` response). In this case we are asserting that the record set must meet these condtions for this stage to be successful:
  - has 144 rows (3 days of 30 minute interval data)  - this is Load Assurance to ensure all records have been received and processed.
  - AND the `SUM` of the `timestamp_null` rule must equal `0` - this is a Data Quality 'completentess' guarantee that can be provided to consumers of the data
  - AND the `SUM` of the `air_temp_null` rule must also equal `0` - this is a Data Quality 'completentess' guarantee that can be provided to consumers of the data
  
1. additionally we can return a message that is added to the logs. in this case we are returning a [JSON](https://en.wikipedia.org/wiki/JSON) formatted string which looks like `{"count":144,"timestamp_null":0,"air_temp_null":0}`. This is very useful when monitoring this job in when it is operational as we can track metrics or set up alerts when certain conditions occur.

In [ ]:
%sqlvalidate name="validate dataset" environments=production,test
SELECT
  COUNT(*) = 144 AND SUM(timestamp_null) = 0 AND SUM(air_temp_null) = 0 AS valid
  ,TO_JSON(
      NAMED_STRUCT(
        'count', COUNT(*),
        'timestamp_null', SUM(timestamp_null),
        'air_temp_null', SUM(air_temp_null)
      )
  ) AS message
FROM (
  SELECT
    CASE
      WHEN timestamp IS NULL THEN 1
      ELSE 0
    END AS timestamp_null
    ,CASE
      WHEN air_temp IS NULL THEN 1
      ELSE 0
    END AS air_temp_null
  FROM weather
) input_table

7/7

valid,message
true,"{""count"":144,""timestamp_null"":0,""air_temp_null"":0}"


### Use the data

At this point we can write arbitrary SQL against the `weather` dataset as a standard [SQL](https://en.wikipedia.org/wiki/SQL) table using the `%sql` command:

In [ ]:
%sql
-- find records where the difference between apparent temperature (feels like) and air temperature correlates with wind speed
SELECT 
  air_temp 
  ,apparent_t
  ,air_temp - apparent_t AS difference_t
  ,wind_spd_kmh
FROM weather
WHERE air_temp > apparent_t

5/5

air_temp,apparent_t,difference_t,wind_spd_kmh
23.90,23.80,0.10,11
17.50,17.10,0.40,9
17.50,16.90,0.60,9
17.60,16.60,1.00,11
17.80,17.20,0.60,9
18.10,17.80,0.30,7
18.60,17.90,0.70,9
18.90,17.30,1.60,13
19.00,17.90,1.10,11
18.80,17.60,1.20,11


### Export the data

As we are confident our data is of good quality (due to passing the [SQLValidate](https://arc.tripl.ai/validate/#sqlvalidate) stage) we can export the data so it can be safely consumed by other people and jobs. 

To do so we can use the [ParquetLoad](https://arc.tripl.ai/load/#parquetload) stage to write the data out to a [Parquet](https://parquet.apache.org/) format which can then can be easily reloaded without losing any data precision using a [ParquetExtract](https://arc.tripl.ai/extract/#parquetextract) stage.

In [ ]:
{
  "type": "ParquetLoad",
  "name": "write out flattened weather dataset",
  "environments": ["production", "test"],
  "inputView": "weather",
  "outputURI": "/home/jovyan/examples/weather/output/"${WMO_STATION_ID}"/weather.parquet",
  "saveMode": "Append"
}

5/5

timestamp,air_temp,apparent_t,delta_t,dewpt,press,wind_spd_kmh,history_product,wmo,refresh_message,_index
2021-03-25 01:00:00Z,26.40,27.10,6.40,16.10,1003.90,7,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,0
2021-03-25 00:30:00Z,26.20,26.90,6.20,16.20,1003.90,7,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,1
2021-03-25 00:00:00Z,25.10,25.40,5.60,16.00,1003.80,9,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,2
2021-03-24 23:30:00Z,23.90,23.80,4.90,15.90,1003.80,11,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,3
2021-03-24 23:00:00Z,21.60,22.40,3.40,15.90,1003.80,6,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,4
2021-03-24 22:30:00Z,21.30,22.00,3.20,16.00,1003.70,7,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,5
2021-03-24 22:00:00Z,18.90,20.10,2.30,15.00,1003.50,2,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,6
2021-03-24 21:30:00Z,18.10,19.50,2.10,14.40,1003.00,0,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,7
2021-03-24 21:00:00Z,17.60,17.70,2.10,13.90,1002.40,6,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,8
2021-03-24 20:30:00Z,17.30,17.40,2.00,13.80,1002.20,6,IDN60901,94768,Issued at 12:02 pm EDT Thursday 25 March 2021,9


### Execute the Job

Now that a job has been built in this notebook it is possible to execute it using the [Arc](https://arc.tripl.ai/) Docker image. Notice that we have defined the `WMO_STATION_ID` parameter using an environment variable which can be easily changed to a different station. This could be scheduled to run periodically to retrieve this data.

```bash
docker run \
--rm \
-v $(pwd)/examples:/home/jovyan/examples:Z \
-e "ETL_CONF_ENV=production" \
-e "WMO_STATION_ID=94768" \
-p 4040:4040 \
ghcr.io/tripl-ai/arc:latest \
bin/spark-submit \
--master local[*] \
--driver-memory 4g \
--class ai.tripl.arc.ARC \
/opt/spark/jars/arc.jar \
--etl.config.uri=file:///home/jovyan/examples/weather/ProcessJSON.ipynb
```